In [1]:
#Import of Python available libraries
import sculib

#mpi = sculib.scu()
#mpi.ip = '10.96.64.10'
sim = sculib.scu()
sim.ip = '10.96.254.85'

In [2]:
def mpi_t622b(scu):
    print('Test 6.2.2b')

    #Revisions:
    #0.1 initial write - could do with some kat sensor retrieval
    #0.2 rewrite for sculib

    #Common setup
    config_name = 'HN_TILT_TEST'

    
    loop1 = 5
    azimuths = [-180, -90, 0, 90, 180]
    az_speed = 3

    elevations = [20, 50, 85]
    el_speed  = 0.1
    
    wait5 = 5
    wait10 = 10
    wait100 = 100
    sampling = 200
    scu.debug = False
    #debug test only remove these
    #'''
    loop1 = 5
    elevations = [20, 50, 85]

    wait5 = 1
    wait10 = 1
    wait100 = 1
    sampling = 5000
    scu.debug = False
    #'''

    #B. Tilt Sensor tests [316-000000-043]
    '''
    (Note that this test is added to check the impact of combined azimuth & elevation motion on the tilt – looking into
    “offset” phenomena seen during the holography tests)
        1) Manually move the Dish to Elevation 50°, Azimuth to -180 and de-activate all axes
        2) Manually measure and record the Pedestal wall temperature at the N/E/S/W positions 
        (consider using iButtons)
        3) Mount the REDS to measuring the pitch angle of the Pedestal (below Azimuth bearing) 
        and aligned with the Azimuth starting position
        4) Start the ACU data recording on the SCU using logging configuration TBD,
        • manually start the REDS data recording (60 seconds @ 1Hz sampling)
        • wait for 60 seconds to reference tilt sensor values
            5) Activate the Azimuth and Elevation axis,
            • wait for 15 seconds to complete the activation process
            6) Command Azimuth to “starting position” -90° at 3°/sec and Elevation to 20°,
            • wait 40 seconds to complete the move
            7) Command Elevation to 85° at 1°/sec,
            • wait 70 seconds to complete the move
            8) Command Azimuth to “starting position” at 3°/sec and Elevation to 50 at 1°/sec,
            • wait 40 seconds to complete the move and stabilise
            9) Deactivate the axis
            • manually start the REDS data recording (60 seconds @ 1Hz sampling)
            • wait for 60 seconds
            10) Repeat steps 5) to 9) in the opposite Azimuth direction
            11) Repeat steps 5 ) to 10) for a total of 5 cycles, 
                each time altering the sequence of the Elevation motion per cycle
        12) Stop the ACU data recording on the SCU
        13) Manually measure and record the Pedestal wall temperature at the N/E/S/W positions (consider using iButtons)
        14) Extract the data on the SCU @ 5Hz sampling
        15) Rename the file to TBD (eg. 6p2p2b_i.csv)
        16) Manually re-positioning the REDS tilt sensor to measure the Pedestal roll angle 
            around the azimuth starting position.
        17) Repeat steps 1) to 16)
        18) Repeat steps 1) to 17) for Azimuth angles -90,0,90, 180
    19) Record the sign convention of the REDS
    20) Download the MeerKAT weather station data for the test period
    '''
    for azimuth in azimuths:
        print("\n Azimuth angle: {} \n".format(azimuth))

        #Step 1
        #Manual move dish to starting position
        print("Manual move to az -180 and el 50")
        scu.activate_dmc()
        scu.wait_duration(wait10*3)
        
        az_angle = azimuth
        scu.abs_azimuth(az_angle, az_speed)
        scu.wait_duration(wait100)
        
        el_angle = elevations[1]
        scu.abs_elevation(el_angle, el_speed)
        scu.wait_duration(wait100*8)
        
        scu.deactivate_dmc()
        scu.wait_duration(wait10*2) 
        #Step 2
        #Record pedestal wall temperatures

        #Step 3
        #Mount Reds
        print("Mounting REDS")
        #print("ACU Deactivated - time to setup REDS")
        #wait = input("PRESS ENTER TO CONTINUE.")
        print("Script continuing")
        
        #Step 4 - start ACU recording on SCI
        if scu.logger_state() == 'RECORDING':
            print('WARNING, already recording - attempting to stop and start a fresh logger')
            scu.stop_logger()  
            scu.wait_duration(wait10)
        if scu.logger_state() == 'STOPPED':
            print('Starting logger with config: {}'.format(config_name))
            scu.start_logger(config_name)
            scu.wait_duration(wait10)

        i=0
        while i < loop1:
            i+=1
            print("Loop 1: {}".format(i))
            print(elevations)
            for az_step in [-90, 90]:
                print("Az step: {}".format(az_step))
                #Step 5
                #Activate az and el
                scu.activate_dmc()
                scu.wait_duration(wait10*3)

                #Step 6
                #Move to next az if not there already?
                az_angle = azimuth + az_step
                scu.abs_azimuth(az_angle, az_speed)
                scu.wait_duration(wait100)
                el_angle = elevations[0]
                scu.abs_elevation(el_angle, el_speed)
                scu.wait_duration(wait100*4)

                #Step 7 - Command Elevation to 85° at 1°/sec,
                el_angle = elevations[2]
                scu.abs_elevation(el_angle, el_speed)
                scu.wait_duration(wait100*8)

                #Step 8
                az_angle = azimuth 
                scu.abs_azimuth(az_angle, az_speed)
                scu.wait_duration(wait100)
                el_angle = elevations[1]
                scu.abs_elevation(el_angle, el_speed)
                scu.wait_duration(wait100*4)


                #Step 9
                #Deactivate the axis wait 60s record the REDS 
                scu.deactivate_dmc()
                scu.wait_duration(wait10*6) 
                #Do you want to wait for it?
                print("ACU Deactivated - time to setup REDS")
                #wait = input("PRESS ENTER TO CONTINUE.")
                print("Script continuing")

                #Step - 10) Repeat steps 5) to 9) in the opposite Azimuth direction

            #Step 11 - Repeat steps 5 ) to 10) for a total of 5 cycles, 
            #??? each time altering the sequence of the Elevation motion per cycle
            elevations = [elevations[1], elevations[2], elevations[0]]
                
        #Step 12 stop recording
        scu.stop_logger()
        scu.wait_duration(wait10)
        print(scu.logger_state())  

        #Step 13 - Manually measure and record the Pedestal wall temperature at the N/E/S/W positions (consider using iButtons)

        #Step 14 - Extract the data on the SCU @ 5Hz sampling
        print("Rename the file to TBD (eg. 6p2p2b_i.csv)")
        scu.save_session('6p2p2b_i_' + str(az_angle), interval_ms = sampling)

        #Step 15 - Rename the file to TBD (eg. 6p2p2b_i.csv)

        #Step 16 - Manually re-positioning the REDS tilt sensor 
        print("ACU Deactivated - time to setup REDS")
        #wait = input("PRESS ENTER TO CONTINUE.")
        print("Script continuing")

        #Step 17 - Repeat steps 1) to 16)
        #Step 18 - Repeat steps 1) to 17) for Azimuth angles -90,0,90, 180
    #Step 19 - Record the sign convention of the REDS
    print("Record the sign convention of the REDS")
    #Step 20 - Download the MeerKAT weather station data for the test period
    print("Download the MeerKAT weather station data for the test period")


In [3]:
#mpi_t622b(mpi)
mpi_t622b(sim)


Test 6.2.2b

 Azimuth angle: -180 

Manual move to az -180 and el 50
activate dmc
wait for 3.0s done *
abs az: -180.000 vel: 3.000
wait for 1.0s done *
abs el: 50.000 vel: 0.100
wait for 8.0s done *
deactivate dmc
wait for 2.0s done *
Mounting REDS
Script continuing
logger state 
WARNING, already recording - attempting to stop and start a fresh logger
stop logger
wait for 1.0s done *
logger state 
Starting logger with config: HN_TILT_TEST
start logger:  HN_TILT_TEST
wait for 1.0s done *
Loop 1: 1
[20, 50, 85]
Az step: -90
activate dmc
wait for 3.0s done *
abs az: -270.000 vel: 3.000
wait for 1.0s done *
abs el: 20.000 vel: 0.100
wait for 4.0s done *
abs el: 85.000 vel: 0.100
wait for 8.0s done *
abs az: -180.000 vel: 3.000
wait for 1.0s done *
abs el: 50.000 vel: 0.100
wait for 4.0s done *
deactivate dmc
wait for 6.0s done *
ACU Deactivated - time to setup REDS
Script continuing
Az step: 90
activate dmc
wait for 3.0s done *
abs az: -90.000 vel: 3.000
wait for 1.0s done *
abs el: 20.000

wait for 3.0s done *
abs az: -180.000 vel: 3.000
wait for 1.0s done *
abs el: 20.000 vel: 0.100
wait for 4.0s done *
abs el: 85.000 vel: 0.100
wait for 8.0s done *
abs az: -90.000 vel: 3.000
wait for 1.0s done *
abs el: 50.000 vel: 0.100
wait for 4.0s done *
deactivate dmc
wait for 6.0s done *
ACU Deactivated - time to setup REDS
Script continuing
Az step: 90
activate dmc
wait for 3.0s done *
abs az: 0.000 vel: 3.000
wait for 1.0s done *
abs el: 20.000 vel: 0.100
wait for 4.0s done *
abs el: 85.000 vel: 0.100
wait for 8.0s done *
abs az: -90.000 vel: 3.000
wait for 1.0s done *
abs el: 50.000 vel: 0.100
wait for 4.0s done *
deactivate dmc
wait for 6.0s done *
ACU Deactivated - time to setup REDS
Script continuing
stop logger
wait for 1.0s done *
logger state 
STOPPED
Rename the file to TBD (eg. 6p2p2b_i.csv)
Attempt export and save of session: last at rate 5000 ms
logger sessions 
export session 
Session id: 149 
Log file location: /home/pkotze/checkouts/python/output/6p2p2b_i_-90-16010

wait for 6.0s done *
ACU Deactivated - time to setup REDS
Script continuing
Loop 1: 4
[20, 50, 85]
Az step: -90
activate dmc
wait for 3.0s done *
abs az: 0.000 vel: 3.000
wait for 1.0s done *
abs el: 20.000 vel: 0.100
wait for 4.0s done *
abs el: 85.000 vel: 0.100
wait for 8.0s done *
abs az: 90.000 vel: 3.000
wait for 1.0s done *
abs el: 50.000 vel: 0.100
wait for 4.0s done *
deactivate dmc
wait for 6.0s done *
ACU Deactivated - time to setup REDS
Script continuing
Az step: 90
activate dmc
wait for 3.0s done *
abs az: 180.000 vel: 3.000
wait for 1.0s done *
abs el: 20.000 vel: 0.100
wait for 4.0s done *
abs el: 85.000 vel: 0.100
wait for 8.0s done *
abs az: 90.000 vel: 3.000
wait for 1.0s done *
abs el: 50.000 vel: 0.100
wait for 4.0s done *
deactivate dmc
wait for 6.0s done *
ACU Deactivated - time to setup REDS
Script continuing
Loop 1: 5
[50, 85, 20]
Az step: -90
activate dmc
wait for 3.0s done *
abs az: 0.000 vel: 3.000
wait for 1.0s done *
abs el: 50.000 vel: 0.100
wait for 4.0s 